In [2]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import rasterio
import numpy as np
import contextily as ctx
from shapely.geometry import Point
from rasterio.plot import show
from rasterio.enums import Resampling
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
train_df = pd.read_csv('danwei_meaned_training_data.csv') #2073 rows

In [4]:
test_df = pd.read_csv('data/final_datasets/final.csv')



In [5]:
test_df = test_df.dropna()

In [6]:
test_df.head()

,type,longitude,latitude,NDVI,NDBI,BSI,bio1,bio2,bio3,bio4,...,distriver2,bulk_density,slope,tri,ph,soc,clay,cec,lidar_elevation_value,0
0,candidate,-13.239949,-54.055652,0.258920,0.253903,0.285881,24.325000,25.033333,22.866667,1743.0,...,41888.893348,124,1.070147,4.049082,51,196,285,170,365.0,other
1,candidate,-12.632771,-53.324514,0.856848,-0.330374,-0.294443,24.629166,25.383333,23.150000,1674.0,...,73665.842083,127,0.166004,0.666667,48,154,292,64,336.0,ADE
2,candidate,-12.352090,-53.265710,0.509402,0.118401,0.131558,25.225000,26.000000,23.750000,1736.0,...,55371.915133,131,0.165912,0.314270,52,157,244,88,288.0,ADE
4,candidate,-13.215377,-53.965133,0.354979,0.227358,0.239771,24.150000,24.933334,22.583334,1689.0,...,49880.052296,127,0.841590,3.154460,51,165,289,105,393.0,ADE
5,candidate,-12.102912,-53.417888,0.827869,-0.215070,-0.186977,24.904167,25.533333,23.616667,1764.0,...,23536.117734,128,0.604063,2.538591,47,113,251,78,318.0,ADE


In [11]:
cleaned_test = test_df.drop(columns=['0', 'type'])
cleaned_test.to_csv('dw_cleaned_test.csv', index=False)

In [7]:
train_df.columns

Index(['type', 'longitude', 'latitude', 'NDVI', 'NDBI', 'BSI', 'bio1', 'bio2',
       'bio3', 'bio4', 'bio5', 'bio6', 'bio7', 'bio8', 'bio9', 'bio10',
       'bio11', 'bio12', 'bio13', 'bio14', 'bio15', 'bio16', 'bio17', 'bio18',
       'bio19', 'distriver1', 'distriver2', 'bulk_density', 'slope', 'tri',
       'ph', 'soc', 'clay', 'cec', 'label', 'lidar_elevation_value'],
      dtype='object')

In [8]:
X = train_df.drop(columns = ['longitude', 'latitude', 'label','type'])
y = train_df['type']

In [31]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [32]:
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV


param_grid = {
    'n_estimators': [200, 250,300],
    'max_depth': [10, 15, 20]
}

# Create a random forest classifier
rf = RandomForestClassifier()

# Use random search to find the best hyperparameters
# rand_search = RandomizedSearchCV(rf, 
#                                  param_distributions = param_dist, 
#                                  n_iter=5, 
#                                  cv=5)

rf = RandomForestClassifier()
grid_search = GridSearchCV(estimator=rf,
                           param_grid=param_grid,
                           cv=5,
                           n_jobs=1,  # use all cores
                           verbose=1)
# Fit the random search object to the data
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=1,
             param_grid={'max_depth': [10, 15, 20],
                         'n_estimators': [200, 250, 300]},
             verbose=1)

In [34]:
best_rf = grid_search.best_estimator_
print('Best hyperparameters:',  grid_search.best_params_) # 15,10, 300

Best hyperparameters: {'max_depth': 10, 'n_estimators': 250}


In [35]:
import joblib
joblib.dump(best_rf, "dwbest_rf_model.pkl")

['dwbest_rf_model.pkl']

In [9]:
import joblib
best_rf = joblib.load("dwbest_rf_model.pkl")

In [14]:
test_x = cleaned_test.drop(columns=['longitude', 'latitude'])

In [15]:
y_pred = best_rf.predict(test_x)

In [16]:
y_proba = best_rf.predict_proba(test_x)

In [17]:
y_pred

array(['other', 'ADE', 'ADE', 'ADE', 'ADE', 'ADE', 'ADE', 'ADE', 'ADE',
       'other', 'ADE', 'ADE', 'ADE', 'ADE', 'ADE', 'ADE', 'ADE', 'ADE',
       'ADE', 'ADE', 'ADE', 'ADE', 'ADE', 'ADE', 'other', 'ADE', 'ADE',
       'ADE', 'ADE', 'ADE', 'ADE', 'ADE', 'ADE', 'ADE', 'ADE', 'ADE',
       'ADE', 'ADE', 'ADE', 'ADE', 'ADE', 'ADE', 'ADE', 'ADE', 'ADE',
       'ADE', 'ADE', 'ADE'], dtype=object)

In [18]:
y_proba

array([[0.38848447, 0.2096    , 0.40191553],
       [0.3922559 , 0.27622857, 0.33151553],
       [0.42006225, 0.2434    , 0.33653775],
       [0.39526225, 0.2386    , 0.36613775],
       [0.3971614 , 0.29021832, 0.31262029],
       [0.40706526, 0.2856812 , 0.30725354],
       [0.40104393, 0.28241832, 0.31653775],
       [0.39984393, 0.27721832, 0.32293775],
       [0.38414161, 0.26936612, 0.34649227],
       [0.34707521, 0.29483784, 0.35808696],
       [0.38831605, 0.3072812 , 0.30440275],
       [0.39088447, 0.30502857, 0.30408696],
       [0.40488748, 0.2992812 , 0.29583132],
       [0.41251304, 0.2562    , 0.33128696],
       [0.37885298, 0.29883784, 0.32230918],
       [0.3882559 , 0.30422857, 0.30751553],
       [0.37324393, 0.26461832, 0.36213775],
       [0.41526225, 0.2466    , 0.33813775],
       [0.37231605, 0.3192812 , 0.30840275],
       [0.42143368, 0.25162857, 0.32693775],
       [0.38414161, 0.26536612, 0.35049227],
       [0.40086225, 0.27162857, 0.32750918],
       [0.

In [ ]:
# outout the result table 

In [19]:
import pandas as pd

class_names = best_rf.classes_

proba_df = pd.DataFrame(y_proba, columns=[f"prob_{cls}" for cls in class_names])

result_df = pd.DataFrame({
    "latitude": cleaned_test["latitude"].values,
    "longitude": cleaned_test["longitude"].values,
    "predicted_type": y_pred
})

# Combine
final_df = pd.concat([result_df, proba_df], axis=1)

In [22]:
final_df

,latitude,longitude,predicted_type,prob_ADE,prob_earthwork,prob_other
0,-54.055652,-13.239949,other,0.388484,0.209600,0.401916
1,-53.324514,-12.632771,ADE,0.392256,0.276229,0.331516
2,-53.265710,-12.352090,ADE,0.420062,0.243400,0.336538
3,-53.965133,-13.215377,ADE,0.395262,0.238600,0.366138
4,-53.417888,-12.102912,ADE,0.397161,0.290218,0.312620
5,-53.277830,-12.309609,ADE,0.407065,0.285681,0.307254
6,-53.361174,-12.257455,ADE,0.401044,0.282418,0.316538
7,-53.427555,-12.078872,ADE,0.399844,0.277218,0.322938
8,-55.061950,-13.065669,ADE,0.384142,0.269366,0.346492
9,-54.942240,-13.137874,other,0.347075,0.294838,0.358087


In [21]:
final_df.to_csv('dw_prediction_results.csv', index=False)